In [1]:
import CMF
import PyPlot; plt = PyPlot;
import Random; Random.seed!(0);
import JLD

## Load datasets

In [2]:
# Constants

K, L = 10, 20
PATH = "/home/anthony/cmf_data/"

runtimes = Dict(
    :maze => 20*60,
    :piano => 30,  # 30
    :drums => 30,  # 30
    :voice => 30  # 30
)
labels = Dict(:hals => "HALS", :mult => "MULT", :anls => "ANLS")
alg_list = [:hals, :mult, :anls]
;

In [3]:
datasets = Dict(
    :maze => CMF.maze(
        path=string(PATH, "Achilles_10252013_sessInfo.mat"),
        epoch=nothing,
        end_time=-1,
        bin_time=5e-3,
        kernel_width=2
    ),
    :piano => CMF.piano(),
    :drums => CMF.drums_mix(),
    :voice => CMF.female_voices()
)
;

## Fit data

In [ ]:
results = Dict()
for key in keys(datasets)
    results[key] = Dict()
    println(string("Fitting ", key))
    
    for alg in alg_list
        results[key][alg] = CMF.fit_cnmf(
            datasets[key], alg=alg,
            K=K, L=L, max_time=runtimes[key], max_itr=Inf
        )
    end
end

Fitting drums
Fitting piano


In [ ]:
JLD.save("./experimental_comparison.jld", "results", results)

## Plot loss histories

In [ ]:
results = JLD.load("./experimental_comparison.jld", "results");

In [ ]:
include("./plot_settings.jl")

# Plot

fig, axes = plt.subplots(2, 2, figsize=set_size(doc_textwidth))
key_list = [:maze, :piano, :drums, :voice]
axes = [axes[1, 1], axes[1, 2], axes[2, 1], axes[2, 2]]
for i in 1:4
    key = key_list[i]
    
    for alg in alg_list
        r = results[key][alg]
        
        if (i == 1)
            label = labels[alg]
        else
            label = ""
        end
        
        axes[i].plot(
            r.time_hist, r.loss_hist,
            label=label
        )
    end
    axes[i].set_title(key, loc="right", pad=-12)
    println(key, size(datasets[key]))

    if (i == 3 || i == 4)
        axes[i].set_xlabel("Time (seconds)")
    end
    if (i == 1 || i == 3)
         axes[i].set_ylabel("Loss")
    end
    plt.show()
end

plt.suptitle("Loss on experimental datasets")
fig.legend(loc="upper left")
plt.savefig("./experimental_comparison.pdf", format="pdf")